# QLoRA Fine-tuning of a BERT SLM for Named Entity Recognition - Token Classification

Token classification assigns a label to individual tokens in a sentence. 

One of the most common token classification tasks is Named Entity Recognition (NER). 

NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

![](https://i.imgur.com/PP1KMBJ.png)

This guide will show you how to:

1. Use QLoRa based PEFT to fine-tune [DistilBERT](https://huggingface.co/distilbert-base-uncased) on the [WNUT 17](https://huggingface.co/datasets/wnut_17) dataset to detect new entities.
   
2. Use your finetuned model for inference.

<Tip>
The task illustrated in this tutorial is supported by the following model architectures:

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[ALBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/albert), [BERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bert), [BigBird](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/big_bird), [BioGpt](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/biogpt), [BLOOM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bloom), [CamemBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/camembert), [CANINE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/canine), [ConvBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/convbert), [Data2VecText](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/data2vec-text), [DeBERTa](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/deberta), [DeBERTa-v2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/deberta-v2), [DistilBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/distilbert), [ELECTRA](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/electra), [ERNIE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/ernie), [ErnieM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/ernie_m), [ESM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/esm), [FlauBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/flaubert), [FNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/fnet), [Funnel Transformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/funnel), [GPT-Sw3](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt-sw3), [OpenAI GPT-2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt2), [GPTBigCode](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt_bigcode), [GPT Neo](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt_neo), [GPT NeoX](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gpt_neox), [I-BERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/ibert), [LayoutLM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/layoutlm), [LayoutLMv2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/layoutlmv2), [LayoutLMv3](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/layoutlmv3), [LiLT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/lilt), [Longformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/longformer), [LUKE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/luke), [MarkupLM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/markuplm), [MEGA](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mega), [Megatron-BERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/megatron-bert), [MobileBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mobilebert), [MPNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mpnet), [Nezha](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nezha), [Nyströmformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nystromformer), [QDQBert](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/qdqbert), [RemBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/rembert), [RoBERTa](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roberta), [RoBERTa-PreLayerNorm](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roberta-prelayernorm), [RoCBert](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roc_bert), [RoFormer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/roformer), [SqueezeBERT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/squeezebert), [XLM](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm), [XLM-RoBERTa](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-roberta), [XLM-RoBERTa-XL](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-roberta-xl), [XLNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlnet), [X-MOD](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xmod), [YOSO](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/yoso)

<!--End of the generated tip-->

</Tip>

In [1]:
import torch
torch.cuda.empty_cache()

## Load WNUT 17 dataset

Start by loading the WNUT 17 dataset from the 🤗 Datasets library:

In [2]:
from datasets import load_dataset

wnut = load_dataset("wnut_17", 
                    trust_remote_code=True)

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

wnut_17.py:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

Then take a look at an example:

In [3]:
wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

Each number in `ner_tags` represents an entity. Convert the numbers to their label names to find out what the entities are:

In [4]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

The letter that prefixes each `ner_tag` indicates the token position of the entity:

- `B-` indicates the beginning of an entity.
- `I-` indicates a token is contained inside the same entity (for example, the `State` token is a part of an entity like
  `Empire State Building`).
- `0` indicates the token doesn't correspond to any entity.

## Preprocess

The next step is to load a DistilBERT tokenizer to preprocess the `tokens` field:

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet and you'll need to set `is_split_into_words=True` to tokenize the words into subwords. For example:

Here you can find which subwords make up a word by looking at the common word IDs

In [6]:
wnut["train"][32]

{'id': '32',
 'tokens': ['I',
  'just',
  'took',
  '"',
  'When',
  'your',
  'mom',
  'makes',
  'you',
  'go',
  'live',
  'with',
  'your',
  'dad',
  'Scooter',
  'Braun',
  ',',
  'your',
  'life',
  '..."',
  'and',
  'got',
  ':',
  'Part',
  '7',
  '!',
  'Try',
  'it',
  ':',
  'http://tinyurl.com/27r2pdz'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  9,
  10,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [7]:
import pandas as pd

example = wnut["train"][32]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
word_ids = tokenized_input.word_ids()

pd.DataFrame({'Tokens' : tokens, 'WordID' : word_ids})

,Tokens,WordID
0,[CLS],NaN
1,i,0.0
2,just,1.0
3,took,2.0
4,"""",3.0
5,when,4.0
6,your,5.0
7,mom,6.0
8,makes,7.0
9,you,8.0


However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function.
3. Labeling .

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

e.g Sergey Brin could have labels of B-person I-Person

however when tokenized it will become Sergey, Br, ##in, so we should have two NER labels in that case, it should be B-Person, I-Person and -100 for Sergey, Br, ##in, we just type in -100 so the model learns to ignore predicting anything different for a subword as anyway it becomes a part of its previous tokens when being considered which are already tagged

In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [9]:
result = tokenize_and_align_labels(wnut['train'][32:33])
result

{'input_ids': [[101, 1045, 2074, 2165, 1000, 2043, 2115, 3566, 3084, 2017, 2175, 2444, 2007, 2115, 3611, 8040, 17206, 2121, 21909, 1010, 2115, 2166, 1012, 1012, 1012, 1000, 1998, 2288, 1024, 2112, 1021, 999, 3046, 2009, 1024, 8299, 1024, 1013, 1013, 4714, 3126, 2140, 1012, 4012, 1013, 2676, 2099, 2475, 17299, 2480, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, -100, -100, 10, 0, 0, 0, 0, -100, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]]}

In [10]:
pd.DataFrame({
    'token': [tokenizer.decode(id) for id in result['input_ids'][0]],
    'ner_id': result['labels'][0]
})

,token,ner_id
0,[CLS],-100
1,i,0
2,just,0
3,took,0
4,"""",0
5,when,0
6,your,0
7,mom,0
8,makes,0
9,you,0


To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [11]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [12]:
tokenized_wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'input_ids': [101,
  1030,
  2703,
  17122,
  2009,
  1005,
  1055,
  1996,
  3193,
  2013,
  2073,
  1045,
  1005,
  1049,
  2542,
  2005,
  2048,
  3134,
  1012,
  3400,
  2110,
  2311,
  1027,
  9686,
  2497,
  1012,
  3492,
  2919,
  4040,
  2182,
  2197,
  3944,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  -100,
  -100,
  0,
  0,
  -100,
  0,
  0,
  0,
  0,
  0,
  0,
  

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric). Seqeval actually produces several scores: precision, recall, F1, and accuracy.

In [14]:
import evaluate

seqeval = evaluate.load("seqeval")

Get the NER labels first, and then create a function that passes your true predictions and true labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the scores:

In [15]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [16]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

Remember we load the same BERT model with quantized weights for QLoRA

In [17]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, BitsAndBytesConfig
import torch

config = BitsAndBytesConfig(
    load_in_4bit=True, # quantize the model to 4-bits when you load it
    bnb_4bit_quant_type="nf4", # use a special 4-bit data type for weights initialized from a normal distribution
    bnb_4bit_use_double_quant=True, # nested quantization scheme to quantize the already quantized weights
    bnb_4bit_compute_dtype=torch.bfloat16, # use bfloat16 for faster computation
    llm_int8_skip_modules=["classifier", "pre_classifier"]
)

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=13, 
    id2label=id2label, 
    label2id=label2id,
    quantization_config=config
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [19]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [20]:
print_trainable_parameters(model)

trainable params: 0 || all params: 45139213 || trainable%: 0.0


In [21]:
model

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (k_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (v_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (out_lin): Linear4bit(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, in

We configure the LoRA settings below just like the previous hands-on demo

In [22]:
from peft import LoraConfig, get_peft_model, TaskType, replace_lora_weights_loftq

config = LoraConfig(
    r=256, # we use a higher rank here as the task is more complex so we want less information loss
    lora_alpha=64,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
    task_type=TaskType.TOKEN_CLS)
 
peft_model = get_peft_model(model, config)
print_trainable_parameters(peft_model)

trainable params: 9447181 || all params: 54586394 || trainable%: 17.30684206764052


At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the seqeval scores and save the training checkpoint.
   
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.

3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [23]:
tokenized_wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1287
    })
})

In [24]:
3394 / 32

106.0625

In [25]:
106 * 6

636

In [27]:
batch_size = 32

# Set up the training arguments for Named Entity Recognition (NER) fine-tuning
training_args = TrainingArguments(
    output_dir="distilbert-ner-qlorafinetune-runs",  # Directory where the model checkpoints and outputs will be saved.
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False}, # suppress warnings
    eval_strategy="steps",                          # Perform evaluation at regular intervals during training.
    save_strategy="steps",                          # Save the model checkpoint at regular intervals.
    learning_rate=4e-4,                             # Higher initial learning rate, as NER tasks may benefit from faster learning.
    logging_steps=20,                               # Log training metrics every 20 steps.
    eval_steps=20,                                  # Perform evaluation every 20 steps.
    save_steps=100,                                 # Save the model checkpoint every 100 steps.
    per_device_train_batch_size=batch_size,         # Batch size per GPU/TPU core/CPU during training.
    per_device_eval_batch_size=batch_size,          # Batch size per GPU/TPU core/CPU during evaluation; reduce if out of memory (OOM) errors occur.
    max_steps=640,                                  # Stop training after 640 total steps.
    weight_decay=0.01,                              # Apply weight decay to reduce overfitting.
    push_to_hub=False,                              # Do not push the model to the Hugging Face Hub after training.
    fp16=True,                                      # Use 16-bit floating point precision to reduce memory usage and speed up training.
    optim="paged_adamw_8bit",                       # Use an 8-bit AdamW optimizer for memory efficiency and faster computation.
)


trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["validation"],
    processing_class=tokenizer, # used to be called tokenizer earlier
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Remember we are training only the LoRA adapter (matrices) and it should take roughly 4-5 mins on a 48 GB GPU

In [28]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
20,1.068600,0.512164,0.000000,0.000000,0.000000,0.920529
40,0.306000,0.478233,0.000000,0.000000,0.000000,0.920529
60,0.316200,0.503947,0.000000,0.000000,0.000000,0.920529
80,0.286700,0.462765,0.000000,0.000000,0.000000,0.920529
100,0.249000,0.349715,0.000000,0.000000,0.000000,0.920529
120,0.275200,0.323173,0.000000,0.000000,0.000000,0.920529
140,0.213400,0.342291,0.600000,0.003589,0.007134,0.920783
160,0.167500,0.298939,0.416838,0.242823,0.306878,0.933944
180,0.179800,0.277783,0.525952,0.181818,0.270222,0.932609
200,0.172100,0.259502,0.556723,0.316986,0.403963,0.939475


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precisi

TrainOutput(global_step=640, training_loss=0.16408095378428697, metrics={'train_runtime': 93.3569, 'train_samples_per_second': 219.373, 'train_steps_per_second': 6.855, 'total_flos': 360160921269624.0, 'train_loss': 0.16408095378428697, 'epoch': 5.981308411214953})

## Save and Load Fine-tuned BERT Named Entity Recognition LoRA Adapter

Parameter-Efficient Fine Tuning (PEFT) methods freeze the pretrained model parameters during fine-tuning and add a small number of trainable parameters (the LoRA adapters) on top of it. 

The adapters are trained to learn task-specific information. 

In this case we trained a LoRA adapter for NER with BERT, let's save it.

Adapters trained with PEFT are also usually an order of magnitude smaller than the full model, making it convenient to share, store, and load and switch them!

In [29]:
trainer.save_model('qlora-distilbert-ner-adapter')

In [30]:
# remove model checkpoints
!rm -rf distilbert-ner-qlorafinetune-runs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Load NER LoRA Adapter into Base Model

We start by loading the base DistilBERT model here

In [31]:
ner_model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", 
    num_labels=13, 
    id2label=id2label, 
    label2id=label2id,
)
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased', fast=True)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now we add in the NER LoRA Adapter to this base model.

So at inference time, each weight matrix in the attention layer (W) will be added with the product of the two corresponding LoRA matrices used to approximate it during the fine-tuning (AxB) i.e __W + AxB__ and these updated weight matrices are used inside the model for inference and predictions. 

In [32]:
ner_model.load_adapter(peft_model_id='qlora-distilbert-ner-adapter',
                       adapter_name='ner')

## Try on sample data

In [33]:
text = "Google Search was invented by Larry Page and Sergey Brin"

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for NER with your model, and pass your text to it:

In [34]:
from transformers import pipeline

classifier = pipeline("ner", model=ner_model, tokenizer=tokenizer,
                      device='cuda')
classifier(text)

Device set to use cuda


[{'entity': 'B-corporation',
  'score': 0.55284154,
  'index': 1,
  'word': 'google',
  'start': 0,
  'end': 6},
 {'entity': 'B-person',
  'score': 0.8707909,
  'index': 6,
  'word': 'larry',
  'start': 30,
  'end': 35},
 {'entity': 'I-person',
  'score': 0.9641062,
  'index': 7,
  'word': 'page',
  'start': 36,
  'end': 40},
 {'entity': 'B-person',
  'score': 0.566822,
  'index': 9,
  'word': 'sergey',
  'start': 45,
  'end': 51},
 {'entity': 'I-person',
  'score': 0.93272555,
  'index': 10,
  'word': 'br',
  'start': 52,
  'end': 54},
 {'entity': 'I-person',
  'score': 0.6995924,
  'index': 11,
  'word': '##in',
  'start': 54,
  'end': 56}]

In [37]:
text = "I am going to San Francisco for a conference organized by Apple and Microsoft"
classifier(text)

[{'entity': 'B-location',
  'score': 0.9541195,
  'index': 5,
  'word': 'san',
  'start': 14,
  'end': 17},
 {'entity': 'I-location',
  'score': 0.79591817,
  'index': 6,
  'word': 'francisco',
  'start': 18,
  'end': 27},
 {'entity': 'B-corporation',
  'score': 0.6386267,
  'index': 12,
  'word': 'apple',
  'start': 58,
  'end': 63},
 {'entity': 'B-corporation',
  'score': 0.6593133,
  'index': 14,
  'word': 'microsoft',
  'start': 68,
  'end': 77}]